***vis_alternatives_selected_question* function**:
* Input: players_info dictionary 
* Displays bar chart showing the alternatives selcted in each multiple-choice question

In [1]:
def vis_alternatives_selected_question (players_info):
    # find full list of alternatives
    alternatives_list = set()
    for player in players_info.keys():
        alternatives_list = alternatives_list.union(set(players_info[player]["alternatives"].keys()))  
    alternatives_list = list(alternatives_list)

    # responses per alternative
    selected_per_alternative = {}
    for alt in alternatives_list:
        selected = []
        for player in players_info.keys():
             if alt in players_info[player]["alternatives"].keys():
                alternative, _ = zip(*players_info[player]["alternatives"][alt])
                selected.extend(list(alternative))
        selected_per_alternative[alt] = Counter(selected)
        
    # Multiple bar chart showing the alternatives selected in each multiple-choice question.

    number_of_plots = len(selected_per_alternative)
    fig, axs = plt.subplots(number_of_plots)
    i = 0
    for key in selected_per_alternative.keys():
        axs[i].bar(range(len(selected_per_alternative[key])), 
                   selected_per_alternative[key].values(), 
                   tick_label=list(selected_per_alternative[key].keys()))
        axs[i].set_title('Responses to '+key)
        axs[i].set(xlabel="Response")
        i += 1

    plt.tight_layout()
    plt.show()